# Maven Taxi Challenge
Four years worth of NYC Taxi Trips to clean, analyze, and visualize using Tableau. The submission will be a Tableau dashboard that will answer:
- average number of trips
- average fare per trip
- average distance traveled
- change in trip volume
- days / times of the week that are the busiest
- popular pick-up and drop-off locations

### Data Review
I will start by gathering the libraries I will need to work with the taxi data

In [7]:
# import libraries
import os
import glob
import pandas as pd

path = os.getcwd()

Merge taxi_trip data into one dataframe

In [15]:
# using glob to get all the csv files in the taxi_trips folder
filepath = path + "/taxi_trips/"
csv_files = glob.glob(os.path.join(filepath, "*.csv"))

# read data from each csv file
taxi_data = [pd.read_csv(file, low_memory=False) for file in csv_files]

# append each file to the dataframe
taxi_data = pd.concat(taxi_data)

taxi_data.head()

/Users/lola/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


,DOLocationID,PULocationID,RatecodeID,VendorID,congestion_surcharge,extra,fare_amount,improvement_surcharge,lpep_dropoff_datetime,lpep_pickup_datetime,mta_tax,passenger_count,payment_type,store_and_fwd_flag,tip_amount,tolls_amount,total_amount,trip_distance,trip_type
0,75,74,1.0,2.0,0.0,0.0,6.5,0.3,2020-01-01 06:52:54.000,2020-01-01 06:47:28.000,0.5,1.0,2.0,N,0.0,0.0,7.3,1.47,1.0
1,75,74,1.0,2.0,0.0,0.0,6.5,0.3,2020-01-01 13:30:43.000,2020-01-01 13:25:34.000,0.5,1.0,2.0,N,0.0,0.0,7.3,1.49,1.0
2,75,74,1.0,2.0,0.0,0.0,6.5,0.3,2020-01-01 14:26:25.000,2020-01-01 14:20:35.000,0.5,1.0,2.0,N,0.0,0.0,7.3,1.31,1.0
3,75,74,1.0,2.0,0.0,0.0,6.5,0.3,2020-01-02 07:03:03.000,2020-01-02 06:56:47.000,0.5,1.0,2.0,N,0.0,0.0,7.3,1.43,1.0
4,75,74,1.0,2.0,0.0,0.0,6.5,0.3,2020-01-02 09:41:02.000,2020-01-02 09:34:46.000,0.5,1.0,2.0,N,0.0,0.0,7.3,1.10,1.0


In [16]:
# count of rows and columns
taxi_data.shape

(28326071, 19)

#### Command Line Shortcuts
Using the command line to do a quick count of each csv file to make sure the total number of rows in the taxi_data dataframe is equal to the total number of rows in all the csv files combined.
- use ls to list files in the current directory
- use cd folder-name/ to change directories
- use wc -l < file-name.csv to count the number of lines in the file (keep in mind one line is the header)

In [17]:
taxi_data.dtypes

DOLocationID               int64
PULocationID               int64
RatecodeID               float64
VendorID                 float64
congestion_surcharge     float64
extra                    float64
fare_amount              float64
improvement_surcharge    float64
lpep_dropoff_datetime     object
lpep_pickup_datetime      object
mta_tax                  float64
passenger_count          float64
payment_type             float64
store_and_fwd_flag        object
tip_amount               float64
tolls_amount             float64
total_amount             float64
trip_distance            float64
trip_type                float64
dtype: object

In [23]:
# missing values in each column
taxi_data.isnull().sum()

DOLocationID                    0
PULocationID                    0
RatecodeID                      0
VendorID                        0
congestion_surcharge     21059703
extra                           0
fare_amount                     0
improvement_surcharge           0
lpep_dropoff_datetime           0
lpep_pickup_datetime            0
mta_tax                         0
passenger_count                 0
payment_type                    0
store_and_fwd_flag              0
tip_amount                      0
tolls_amount                    0
total_amount                    0
trip_distance                   0
trip_type                     365
dtype: int64

### Data Cleaning
The raw data has some issues, so instructions / assumptions were given to clean and prep the data

In [24]:
# values in the "store and forward" column
taxi_data["store_and_fwd_flag"].unique()

array(['N'], dtype=object)

In [22]:
# Keep trips that were not sent via "store and forward"
taxi_data = taxi_data[taxi_data["store_and_fwd_flag"] == "N"]
taxi_data.head()

,DOLocationID,PULocationID,RatecodeID,VendorID,congestion_surcharge,extra,fare_amount,improvement_surcharge,lpep_dropoff_datetime,lpep_pickup_datetime,mta_tax,passenger_count,payment_type,store_and_fwd_flag,tip_amount,tolls_amount,total_amount,trip_distance,trip_type
0,75,74,1.0,2.0,0.0,0.0,6.5,0.3,2020-01-01 06:52:54.000,2020-01-01 06:47:28.000,0.5,1.0,2.0,N,0.0,0.0,7.3,1.47,1.0
1,75,74,1.0,2.0,0.0,0.0,6.5,0.3,2020-01-01 13:30:43.000,2020-01-01 13:25:34.000,0.5,1.0,2.0,N,0.0,0.0,7.3,1.49,1.0
2,75,74,1.0,2.0,0.0,0.0,6.5,0.3,2020-01-01 14:26:25.000,2020-01-01 14:20:35.000,0.5,1.0,2.0,N,0.0,0.0,7.3,1.31,1.0
3,75,74,1.0,2.0,0.0,0.0,6.5,0.3,2020-01-02 07:03:03.000,2020-01-02 06:56:47.000,0.5,1.0,2.0,N,0.0,0.0,7.3,1.43,1.0
4,75,74,1.0,2.0,0.0,0.0,6.5,0.3,2020-01-02 09:41:02.000,2020-01-02 09:34:46.000,0.5,1.0,2.0,N,0.0,0.0,7.3,1.10,1.0


In [31]:
taxi_data["payment_type"].unique()

array([2., 1.])

In [30]:
# keep trips where payment type is equal to 1 - Credit Card or 2 - Cash
taxi_data = taxi_data[taxi_data["payment_type"].isin([1.0, 2.0])]

In [32]:
taxi_data["RatecodeID"].unique()

array([ 1.,  5.,  2.,  3.,  4.,  6., 99.])

In [33]:
# keep trips where RatecodeID is equal to 1 - Standard Rate
taxi_data = taxi_data[taxi_data["RatecodeID"] == 1.0]
taxi_data.head()

,DOLocationID,PULocationID,RatecodeID,VendorID,congestion_surcharge,extra,fare_amount,improvement_surcharge,lpep_dropoff_datetime,lpep_pickup_datetime,mta_tax,passenger_count,payment_type,store_and_fwd_flag,tip_amount,tolls_amount,total_amount,trip_distance,trip_type
0,75,74,1.0,2.0,0.0,0.0,6.5,0.3,2020-01-01 06:52:54.000,2020-01-01 06:47:28.000,0.5,1.0,2.0,N,0.0,0.0,7.3,1.47,1.0
1,75,74,1.0,2.0,0.0,0.0,6.5,0.3,2020-01-01 13:30:43.000,2020-01-01 13:25:34.000,0.5,1.0,2.0,N,0.0,0.0,7.3,1.49,1.0
2,75,74,1.0,2.0,0.0,0.0,6.5,0.3,2020-01-01 14:26:25.000,2020-01-01 14:20:35.000,0.5,1.0,2.0,N,0.0,0.0,7.3,1.31,1.0
3,75,74,1.0,2.0,0.0,0.0,6.5,0.3,2020-01-02 07:03:03.000,2020-01-02 06:56:47.000,0.5,1.0,2.0,N,0.0,0.0,7.3,1.43,1.0
4,75,74,1.0,2.0,0.0,0.0,6.5,0.3,2020-01-02 09:41:02.000,2020-01-02 09:34:46.000,0.5,1.0,2.0,N,0.0,0.0,7.3,1.10,1.0


In [34]:
taxi_data.isnull().sum()

DOLocationID                    0
PULocationID                    0
RatecodeID                      0
VendorID                        0
congestion_surcharge     20359525
extra                           0
fare_amount                     0
improvement_surcharge           0
lpep_dropoff_datetime           0
lpep_pickup_datetime            0
mta_tax                         0
passenger_count                 0
payment_type                    0
store_and_fwd_flag              0
tip_amount                      0
tolls_amount                    0
total_amount                    0
trip_distance                   0
trip_type                     354
dtype: int64

In [35]:
# checking if any unknown zones (264, 265) appear in the pickup location
taxi_data["PULocationID"].unique()

array([ 74,  41,  75,  42,  55, 116,  82,  92,  95, 130,  33,  25, 127,
       112,  97, 196, 146, 260,  80, 227, 108, 166,   7,  18, 129,  49,
       174,  29, 247,  65,  66, 223, 134,  52, 145, 243, 242,  61, 173,
        16, 181,  14, 226, 205,  69,  28, 244, 254, 147,  40,  94,  39,
       167, 159, 131, 179, 225,  26, 197, 228,  93, 136,  78,  24, 165,
       216, 182,  83,  37, 152,   3, 193, 121, 119, 248, 168, 241,  17,
        89,  43,  32,  86,  47, 220,  76,  19, 188, 210, 122,  72, 102,
        21, 263, 177, 189, 175,  60, 235,  20, 157, 191, 192, 236,  98,
       208,  73,  56, 215, 265,  10, 258, 101, 250, 217, 106, 200,  91,
        53, 135, 155, 124,   9,  35, 120,  70,  34, 264,  62, 218, 132,
       171,  81,  54, 160, 219, 213, 149, 253, 203, 257, 169, 212, 115,
        15, 206,  22, 117,  77, 128,  67, 256, 133, 150, 195,   8, 139,
        63,  85, 154, 185, 221,  71, 252, 126,  11,  64,  96, 190, 138,
       255,  59,  57,  51,  46, 198, 123, 153, 222,  36, 180, 18

In [36]:
# checking if any unknown zones (264, 265) appear in the dropoff location
taxi_data["DOLocationID"].unique()

array([ 75,  74,  41,  42, 166, 152, 168, 151, 116, 244,  24,  43,  69,
       159, 247, 243, 238, 126, 127, 263, 119, 167, 235, 236,  47, 169,
       250, 182, 120, 183, 265,  78, 220, 259, 248, 147, 153,  18, 241,
       136, 212, 213,  20, 242,  31,  94,   3,  81, 174, 264, 185,  46,
       208, 254,  58, 194, 138, 128,  59,  32, 240,  60,  51,  93, 200,
       184, 262,  64,  68, 163, 143, 140,  48,  82, 164, 146, 129, 142,
       233,  83,   7, 132, 144, 137, 237, 196, 234,  80,  91,  28, 223,
        37,  86,  71, 149, 181,  22, 155, 226, 173, 216,  97,  61,   9,
       108, 252, 191, 134, 130, 205, 197,  17, 145, 193,  33,  21,  76,
       227, 171, 135,  55, 179,  63,  29,  35,  92,  89, 256,  19,  26,
        25,  16,  14, 222,  70, 124,  11,  53, 180, 198, 225,  65, 101,
       217, 121, 215, 106, 202,  49,  95, 112, 255,  72, 178, 165,  56,
       177,  85,  15, 203, 189, 123, 160,   8,  62, 232,  34, 122, 188,
        39, 157, 260, 258,  36,  67,  77, 228, 209,  66, 131,  9

In [38]:
taxi_data["passenger_count"].value_counts()

1.0    22329627
2.0     2027291
5.0      896542
6.0      466095
3.0      432914
4.0      147786
0.0       23175
9.0           3
7.0           1
Name: passenger_count, dtype: int64

In [ ]:
# for rides with no passengers, replace with 1
taxi_data = taxi_data["passenger_count"].replace(0,1)